# Python for Automation - Assignment 2

## Submitted by: Antonio Tasson
## Assignment Objectives

The objectives of this assignment are to test your understanding of the solution development practices with Python, including:

* Working with classes
* Working with functions
* Working with data files


## Assignment Instructions

For each of the questions below, write Python code to produce the answer. Be sure to run your code and display the output.

To submit the assignment, save your Jupyter notebook and submit the file in Quercus.

### Question 1 (5 marks):

* A company which started in 2020 has asked you to build an HR program to hold information about its employees. You've been given the following requirements: 
    - Create a new class called 'Employees' which holds the following information about each employee: year_onboarded, name, assigned_email, department, manager, current_salary, and assigned_software.
        - The email should take the format of "name@company.com". This should be an auto-populated field.
        - The class should also have methods to return information relevant to HR or IT. It should include, at least:
            - The count of softwares the employee has access to
            - How many years the employee has worked in the company since its inception
            - The employee's raise and new salary at Year End, assuming a 5% increase rate


* Feel free to add any additional attributes or methods you think are necessary.


* Create a few test employees and demonstrate how the attributes and methods defined work.

In [ ]:
from datetime import date

COMPANY_START_YEAR = 2020

def name_to_email(name: str, domain: str = "abc.com") -> str:
    handle = name.strip().lower()
    handle = handle.replace("'", "").replace("`", "").replace("’", "")
    handle = handle.replace("&", "and")
    handle = ".".join(handle.split())  # collapse any whitespace to single dot
    return f"{handle}@{domain}"

class Employees:
    def __init__(self, year_onboarded, name, department, manager, current_salary, assigned_software=None):
        self.year_onboarded = year_onboarded
        self.name = name
        self.department = department
        self.manager = manager
        self.current_salary = float(current_salary)
        self.assigned_software = assigned_software if assigned_software else []
        # auto-generate email
        self.assigned_email = name_to_email(name)

    def software_count(self):
        return len(self.assigned_software)

    def years_worked_since_inception(self, as_of_year=None):
        yr = as_of_year or date.today().year
        start = max(COMPANY_START_YEAR, self.year_onboarded)
        return max(0, yr - start)

    def list_software(self):
        return list(self.assigned_software)

    def year_end_raise(self, rate=0.05):
        raise_amount = round(self.current_salary * rate, 2)
        new_salary = round(self.current_salary + raise_amount, 2)
        return raise_amount, new_salary

    def __repr__(self):
        return (f"Employees(name={self.name!r}, email={self.assigned_email!r}, "
                f"dept={self.department!r}, manager={self.manager!r}, "
                f"salary={self.current_salary:,.2f}, "
                f"software={self.assigned_software})")

In [5]:
e1 = Employees(2020, "John Smith", "Engineering", "Jan Anderson", 95000, ["AutoCAD", "VSCode", "MSExcel"])

print(e1.assigned_email)                      # alice.johnson@company.com
print(e1.software_count())                     # 3
print(e1.years_worked_since_inception())       # depends on current year
print(e1.year_end_raise())                     # (4750.0, 99750.0)
print(e1.list_software()) 

john.smith@abc.com
3
5
(4750.0, 99750.0)
['AutoCAD', 'VSCode', 'MSExcel']


### Question 2 (10 marks):

* To onboard new employees, the company has a process by which HR provides a csv file with the information captured in the class above, includng: Year Onboarded, Name, Department, Manager, and Current Salary. The Manager of the employee provides a separate csv file listing the software requirements for each employee. Now that you have defined the class to hold the relevant data, create a program which can do the following:

- Take the stated csv files as inputs and assign data for each employee to the class
- Produce a single output (e.g., a csv file) which can be given to IT to assign software to all new hires

In [47]:
from datetime import date
import csv

COMPANY_START_YEAR = 2020

def name_to_email(name: str, domain: str = "abc.com") -> str:
    handle = name.strip().lower()
    handle = handle.replace("'", "").replace("`", "").replace("’", "")
    handle = handle.replace("&", "and")
    handle = ".".join(handle.split())  # collapse any whitespace to single dot
    return f"{handle}@{domain}"

class Employees:
    def __init__(self, year_onboarded, name, department, manager,
                 current_salary, assigned_software=None):
        self.year_onboarded = int(year_onboarded)
        self.name = name
        self.department = department
        self.manager = manager
        self.current_salary = float(current_salary)
        self.assigned_software = assigned_software if assigned_software else []
        self.assigned_email = name_to_email(name)

    def software_count(self):
        return len(self.assigned_software)

    def list_software(self):
        return list(self.assigned_software)

    def years_worked_since_inception(self, as_of_year=None):
        yr = as_of_year or date.today().year
        start = max(COMPANY_START_YEAR, self.year_onboarded)
        return max(0, yr - start)

    def year_end_raise(self, rate=0.05):
        raise_amount = round(self.current_salary * rate, 2)
        new_salary = round(self.current_salary + raise_amount, 2)
        return raise_amount, new_salary

    @classmethod
    def from_csv(cls, filename):        
        employees = []
        with open(filename, newline="", encoding="utf-8-sig") as f:
            reader = csv.DictReader(f)
            for row in reader:                
                employees.append(
                    cls(
                        year_onboarded=row["Year Onboarded"],
                        name=row["Name"],
                        department=row["Department"],
                        manager=row["Manager"],
                        current_salary=row["Current Salary"]
                    )
                )
        return employees

    @staticmethod
    def assign_department_software(employees, access_file):
        dept_access = {}
        with open(access_file, newline="", encoding="utf-8-sig") as f:
            reader = csv.DictReader(f)
            for row in reader:
                access_list = [a.strip() for a in row["Access"].split(",")]
                dept_access[row["Department"]] = access_list

        for emp in employees:
            if emp.department in dept_access:
                for sw in dept_access[emp.department]:
                    if sw not in emp.assigned_software:
                        emp.assigned_software.append(sw)

    @staticmethod
    def export_software_assignments(employees, filename):
        with open(filename, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["Name", "Email", "Department", "Manager", "Software"])
            count = 0
            for emp in employees:
                software_sorted = sorted(set(emp.assigned_software))
                software_str = ", ".join(software_sorted)
                writer.writerow([emp.name, emp.assigned_email, emp.department, emp.manager, software_str])
                count += 1
        return count

    def __repr__(self):
        return (f"Employees(name={self.name!r}, email={self.assigned_email!r}, "
                f"dept={self.department!r}, manager={self.manager!r}, "
                f"salary={self.current_salary:,.2f}, "
                f"software={self.assigned_software})")


In [48]:
# Load new hires from file newhires.csv
emps = Employees.from_csv("newhires.csv")

# Assign software by department
Employees.assign_department_software(emps, "access.csv")

# Export a single file for IT
rows = Employees.export_software_assignments(emps, "software_assignments_for_it.csv")


In [49]:
import csv

def list_newhires(filename="software_assignments_for_it.csv"):    
    hires = []
    with open(filename, newline="", encoding="utf-8-sig") as f:
        reader = csv.DictReader(f)
        for row in reader:
            hires.append(row)
    return hires

# Example usage:
for hire in list_newhires():
    print(hire)

{'Name': 'John', 'Email': 'john@abc.com', 'Department': 'Marketing', 'Manager': 'Stephanie', 'Software': 'MS Office, Salesforce'}
{'Name': 'Amy', 'Email': 'amy@abc.com', 'Department': 'Marketing', 'Manager': 'Stephanie', 'Software': 'MS Office, Salesforce'}
{'Name': 'Sidra', 'Email': 'sidra@abc.com', 'Department': 'Finance', 'Manager': 'Roger', 'Software': 'MS Office, Oracle'}
{'Name': 'Naveen', 'Email': 'naveen@abc.com', 'Department': 'Data', 'Manager': 'Matthias', 'Software': 'AWS, MS Office, Python'}
{'Name': 'Delilah', 'Email': 'delilah@abc.com', 'Department': 'Technology', 'Manager': 'Matthias', 'Software': 'AWS, MS Office, Oracle, Python, Salesforce'}
